In [12]:
from plyfile import PlyData, PlyElement

In [13]:
data = PlyData.read('PLY/target.ply')

In [14]:
data

PlyData((PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float'), PlyProperty('texture_u', 'float'), PlyProperty('texture_v', 'float')), count=22474, comments=[]), PlyElement('face', (PlyListProperty('vertex_indices', 'uchar', 'int'),), count=43198, comments=[])), text=False, byte_order='<', comments=[u'VCGLIB generated'], obj_info=[])

In [15]:
X=data['vertex']['x']

In [16]:
Y=data['vertex']['y']

In [17]:
Z=data['vertex']['z']

In [18]:
from mayavi import mlab

In [19]:
import numpy as np
pts = mlab.points3d(X, Y, Z, Z)

In [20]:
mesh = mlab.pipeline.delaunay2d(pts)

In [21]:
pts.remove()

# Draw a surface based on the triangulation
surf = mlab.pipeline.surface(mesh)

# Simple plot.
mlab.xlabel("x")
mlab.ylabel("y")
mlab.zlabel("z")
mlab.show()

In [11]:
import numpy as np
from mayavi.mlab import *

def test_triangular_mesh():
    """An example of a cone, ie a non-regular mesh defined by its
        triangles.
    """
    n = 8
    t = np.linspace(-np.pi, np.pi, n)
    z = np.exp(1j * t)
    x = z.real.copy()
    y = z.imag.copy()
    z = np.zeros_like(x)

    triangles = [(0, i, i + 1) for i in range(1, n)]
    x = np.r_[0, x]
    y = np.r_[0, y]
    z = np.r_[1, z]
    t = np.r_[0, t]

    return triangular_mesh(x, y, z, triangles, scalars=t)

In [12]:
s=test_triangular_mesh()

In [13]:
mlab.show()

In [14]:
len(X)

22474

In [22]:
import os
from os.path import join

# Enthought library imports
from mayavi import mlab
mlab.init_notebook()
# ### Download the bunny data, if not already on disk ############################
# if not os.path.exists('bunny.tar.gz'):
#     # Download the data
#     try:
#         from urllib import urlopen
#     except ImportError:
#         from urllib.request import urlopen
#     print("Downloading bunny model, Please Wait (3MB)")
#     opener = urlopen(
#                 'http://graphics.stanford.edu/pub/3Dscanrep/bunny.tar.gz')
#     open('bunny.tar.gz', 'wb').write(opener.read())

# # Extract the data
# import tarfile
# bunny_tar_file = tarfile.open('bunny.tar.gz')
# try:
#     os.mkdir('bunny_data')
# except:
#     pass
# bunny_tar_file.extractall('bunny_data')
# bunny_tar_file.close()

# # Path to the bunny ply file
# bunny_ply_file = join('bunny_data', 'bunny', 'reconstruction', 'bun_zipper.ply')

# Render the bunny ply file
mlab.pipeline.surface(mlab.pipeline.open("PLY/target.ply"))
mlab.show()

# import shutil
# shutil.rmtree('bunny_data')

Notebook initialized with x3d backend.


In [1]:
import os
from os.path import join

# Enthought library imports
from mayavi import mlab

In [2]:
data=mlab.pipeline.open("PLY/target.ply")

In [4]:
surf=mlab.pipeline.surface(mlab.pipeline.open("PLY/target.ply"))

In [5]:
surf.actor.enable_texture=True

In [1]:
from osgeo import gdal
from tvtk.api import tvtk
from mayavi import mlab
import Image

In [36]:
im1 = Image.open("Checkerboard.jpg")

In [37]:
im2 = im1.rotate(90)
im2.save("tmp/Checkerboard.jpg")

In [38]:
bmp1 = tvtk.JPEGReader(file_name="tmp/Checkerboard.jpg")

In [39]:
my_texture=tvtk.Texture()
my_texture.interpolate=0

In [40]:
# my_texture.set_input(0,bmp1.get_output())
#tvtk.configure_input(my_texture, bmp1)
my_texture=tvtk.Texture(input_connection=bmp1.output_port, interpolate=0)

In [43]:
surf=mlab.pipeline.surface(mlab.pipeline.open("PLY/target.ply"))

In [ ]:
surf.actor.enable_texture = True
surf.actor.tcoord_generator_mode = 'plane'
surf.actor.actor.texture = my_texture
#mlab.show()
mlab.show()